---
layout: post
title: python 贷款利息计算公式
category: finance
tags: [finance, python]
---

# 固定支付贷款

固定支付贷款(fixed-payment loan) 也称为分期偿还贷款(fully amortized loan)。
贷款人向借款人提供一定数量的资金，在约定的若干年度内，借款人每个期限偿还固定的金额，其中包括本金，也包括利息。
例如，你借款金额为1000元，每年偿还126元，还款年限为25年。
分期贷款与抵押贷款，如车贷、房贷，通常都属于这种类型。

固定支付贷款相关的几个变量为：

* 贷款金额LV (loan value)
* 贷款利率i (interest rate)
* 贷款年限n
* 每期还款金额FP (fixed-payment)

我们把每期偿还的金额FP折算成等价的现值，折现率等于贷款利率。然后令贷款金额等于它的现值便可得出以下公式：

$$
LV = \frac{FP}{1+i} + \frac{FP}{(1+i)^2} + \frac{FP}{(1+i)^3} + \cdots + \frac{FP}{(1+i)^n}
$$


假设你需要购买一栋房子，需要向银行贷款 10 万元贷款，从银行贷款的利率为 7%, 你想要在 20 年还完贷款，每年需要付多少钱呢？

我们把每期还款金额折现后求和，应该等于贷款金额，得出：

$$
100000 = \frac{FP}{1+0.07} + \frac{FP}{(1+0.07)^2} + \frac{FP}{(1+0.07)^3} + \cdots + \frac{FP}{(1+0.07)^20}
$$

其中 n = 贷款年数 = 20, LV = 贷款金额 = 10万, i = 贷款利率 = 0.07

求解上述公式得出 FP = 每年还款金额 = 9439.29 元




# 等比公式

我们知道等比数列求和：

$$
S_n = a_1 + a_1 q + a_1 q^2 + \cdots + a_1 q^{n-1}
$$

两边乘以公比q得：

$$
q S_n = a_1 q + a_1 q^2 + a_1 q^3 + \cdots + a_1 q^{n}
$$

上述两式相减得出:

$$
(1-q)S_n = a_1 - a_1 q^{n}
$$

所以,当 q 不等于 1 时，可得到：

$$
S_n = a_1\frac{1 - q^n}{1 - q}
$$

# 公式化简

$$
LV = \frac{FP}{1+i} + \frac{FP}{(1+i)^2} + \frac{FP}{(1+i)^3} + \cdots + \frac{FP}{(1+i)^n}
$$

公式的右边是一个等比数列求和公式，我们令：

* $\displaystyle a_1 = \frac{FP}{(1+i)^n}$
* $q = 1 + i$

把等比求和带入折现公式中，得到以下结果：

$$
\begin{align}
LV & = \frac{FP}{(1+i)^n}  \frac{1 - (1+i)^n}{1 - (1+i)} \\
   & = \frac{FP}{(1+i)^n}  \frac{(1+i)^n - 1}{i} \\
\end{align}
$$

一般情况下，我们根据贷款利率、贷款金额、贷款年限，求得每期还款金额。

$$
FP = LV \frac{i (1+i)^n}{(i+i)^n - 1}
$$

下面我们使用python定义一个函数计算上述公式：


In [5]:
%matplotlib notebook
import numpy as np

def calc_pv(lv, i, n):
    '''
    计算固定支付贷款每期还款金额
    @lv : 贷款金额
    @i  : 贷款利率
    @n  : 贷款期数
    '''
    qn = np.power(1 + i, n)
    fp = qn * lv * i
    fp = fp / (qn - 1)
    return fp

# 按年付款
print("贷款10万, 利率0.07, 20年还清，每年需还款: %f 元" % calc_pv(100000, 0.07, 20))

# 按月付款
print('若按月还款的话，每月需还款 %f 元' % calc_pv(100000, 0.07/12, 20 * 12))

贷款10万, 利率0.07, 20年还清，每年需还款: 9439.292574 元
若按月还款的话，每月需还款 775.298936 元


# 按月付款

按月付款与按年付款的区别仅仅在于利率和还款期数的区别。

* 月利率等于年利率除以12
* 还款期数n等于年数乘以12。

我们房贷一般是按月还款，我们可以自己根据上述公式或者函数计算出每月还款金额。

当然，我们也可以使用[招行个人贷款计算器](http://www.cmbchina.com/CmbWebPubInfo/Cal_Loan_Per.aspx?chnl=dkjsq)计算每月还款金额，其结果跟我们计算的是一样的。

# 利率计算

当我们知道贷款金额，每月还款金额，贷款年数时，我们还可以通过公式计算出银行的贷款利率。

如果使用 python 的话，可以使用 [numpy.irr](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.irr.html) 函数。

下面我们使用代码计算利率：

In [3]:
import numpy as np

def calc_irr(lv, fp, n):
    values = np.full(n + 1, fp)
    values[0] = -lv
    return np.irr(values)

print('如果贷款10万，每年还 9439.29 元，贷款20年，则贷款利率为: ', calc_irr(100000, 9439.29, 20))

如果贷款10万，每年还 9439.29 元，贷款20年，则贷款利率为:  0.06999996491048943


# 参考

* [贷款基准利率](http://www.cmbchina.com/CmbWebPubInfo/CDRate.aspx?chnl=cdrate)
* [招行个人贷款计算器](http://www.cmbchina.com/CmbWebPubInfo/Cal_Loan_Per.aspx?chnl=dkjsq)